# Food Identification mini

In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from shutil import copytree, rmtree

Init Plugin
Init Graph Optimizer
Init Kernel


In [12]:
def dataset_mini(food_list, src, dest):
    if os.path.exists(dest):
        rmtree(dest)
    os.makedirs(dest)
    for food_item in food_list :
        print("Copying images into",food_item)
        copytree(os.path.join(src,food_item), os.path.join(dest,food_item))

In [13]:
food_list = ['donuts','pizza','samosa']
src_train = '../Data/Train'
dest_train = '../Data/Train_mini'
src_test = '../Data/Test'
dest_test = '../Data/Test_mini'

In [9]:
print("Creating train data folder with new classes")
dataset_mini(food_list, src_train, dest_train)
print("Creating test data folder with new classes")
dataset_mini(food_list, src_test, dest_test)

Creating train data folder with new classes
Copying images into donuts
Copying images into pizza
Copying images into samosa
Creating test data folder with new classes
Copying images into donuts
Copying images into pizza
Copying images into samosa


In [14]:
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers

K.clear_session()
n_classes = 3
img_width, img_height = 299, 299
train_data_dir = dest_train
validation_data_dir = dest_test
nb_train_samples = 2250
nb_validation_samples = 750
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


inception = InceptionV3(weights='imagenet', include_top=False)
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(3,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_3class.log')

history = model.fit_generator(train_generator,
                              steps_per_epoch = nb_train_samples // batch_size,
                              validation_data=validation_generator,
                              validation_steps=nb_validation_samples // batch_size,
                              epochs=30,
                              verbose=1,
                              callbacks=[csv_logger, checkpointer])

model.save('model_trained_3class.hdf5')

Found 2250 images belonging to 3 classes.
Found 750 images belonging to 3 classes.
Epoch 1/30


2021-09-25 16:46:47.904709: W tensorflow/core/framework/op_kernel.cc:1755] Invalid argument: ValueError: callback pyfunc_2 is not found
Traceback (most recent call last):

  File "/opt/homebrew/Caskroom/miniforge/base/envs/data-science/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_2 is not found


2021-09-25 16:46:47.904784: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: Invalid argument: ValueError: callback pyfunc_2 is not found
Traceback (most recent call last):

  File "/opt/homebrew/Caskroom/miniforge/base/envs/data-science/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_2 is not found


	 [[{{node PyFunc}}]]
2021-09-25 16:46:49.055784: I tensorflow/core/grapple

140/140 [==============================] - ETA: 0s - loss: 0.9930 - accuracy: 0.5555

2021-09-25 16:48:46.842527: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


140/140 [==============================] - 129s 904ms/step - loss: 0.9930 - accuracy: 0.5555 - val_loss: 0.6456 - val_accuracy: 0.8818

Epoch 00001: val_loss improved from inf to 0.64560, saving model to best_model_3class.hdf5
Epoch 2/30
140/140 [==============================] - 127s 906ms/step - loss: 0.6638 - accuracy: 0.8048 - val_loss: 0.3898 - val_accuracy: 0.9470

Epoch 00002: val_loss improved from 0.64560 to 0.38982, saving model to best_model_3class.hdf5
Epoch 3/30
140/140 [==============================] - 127s 909ms/step - loss: 0.4802 - accuracy: 0.8675 - val_loss: 0.2779 - val_accuracy: 0.9511

Epoch 00003: val_loss improved from 0.38982 to 0.27789, saving model to best_model_3class.hdf5
Epoch 4/30
140/140 [==============================] - 125s 892ms/step - loss: 0.3821 - accuracy: 0.8926 - val_loss: 0.2149 - val_accuracy: 0.9606

Epoch 00004: val_loss improved from 0.27789 to 0.21494, saving model to best_model_3class.hdf5
Epoch 5/30
140/140 [===========================

In [15]:
class_map_3 = train_generator.class_indices
class_map_3

{'donuts': 0, 'pizza': 1, 'samosa': 2}

In [4]:
from tensorflow.keras.models import load_model
model = load_model('../Model/model_trained_3class.hdf5', compile=False)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('../Model/model.tflite', 'wb') as f:
    f.write(tflite_model)

Metal device set to: Apple M1


2021-09-25 20:57:23.807119: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-09-25 20:57:23.807206: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


2021-09-25 20:57:28.655663: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/qj/ky4j5wv51qn6q4zgqgbxydbw0000gn/T/tmpa257hg3o/assets


2021-09-25 20:57:35.675098: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-09-25 20:57:35.675161: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-09-25 20:57:35.675308: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-09-25 20:57:35.675322: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-09-25 20:57:35.675439: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-09-25 20:57:35.682554: I tensorflow/core/grappler